<center><h1>Saravanan_Praveen_HW3</h1></center>
<br>
<br>

Name: Praveen Saravanan
<br>
Github Username: Michael-on-the-Microphone
<br>
USC ID: 7939976839

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [1]:
import os
import re
import shutil
from pathlib import Path
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd

Get the AReM Data Set

In [2]:
nb_dir = Path().resolve() 
DATA_ROOT = nb_dir.parent / "data" / "AReM"        
TRAIN_ROOT = DATA_ROOT.parent / "train"
TEST_ROOT  = DATA_ROOT.parent / "test"

DATASET_RE = re.compile(r"dataset(\d+)\.csv$", re.I)

### (b) Test and Train Data

In [3]:
def dataset_id(path: Path) -> int | None:
    m = DATASET_RE.search(path.name)
    return int(m.group(1)) if m else None

def is_test_file(activity: str, ds_id: int) -> bool:
    if activity in {"bending1", "bending2"}:
        return ds_id in {1, 2}
    return ds_id in {1, 2, 3}

def copy_to_split(src: Path, dest_root: Path):
    """Copy file to dest_root/<activity>/"""
    dest_dir = dest_root / src.parent.name
    dest_dir.mkdir(parents=True, exist_ok=True)
    shutil.copy2(src, dest_dir / src.name)  

took a lil help from gpt to make a sorting snippet

In [4]:
def split():
    for csv in DATA_ROOT.rglob("dataset*.csv"):
        ds_id = dataset_id(csv)
        if ds_id is None:
            continue  

        activity = csv.parent.name.lower()

        target_root = TEST_ROOT if is_test_file(activity, ds_id) else TRAIN_ROOT
        copy_to_split(csv, target_root)

    print(f"Done!  Train files in: {TRAIN_ROOT.resolve()}")
    print(f"        Test files in: {TEST_ROOT.resolve()}")

# split()

In [5]:
def load_csv(path: Path) -> pd.DataFrame:
    """
    AReM files have 7 columns: one timestamp + 6 sensor channels.
    Some versions include 4 comment rows starting with '#'.
    This loader skips comment lines automatically.
    """
    return pd.read_csv(
        path,
        header=None,
        comment="#",
        names=["time","avg_rss12", "var_rss12","avg_rss13", "var_rss13","avg_rss23", "var_rss23"],
    )

### (c) Feature Extraction

#### i. Research

### Basic Time-Domain Features

Minimum / Maximum – The lowest and highest values within the window 

Mean – Average value, representing central tendency 

Median – Middle value, providing a robust statistic against outliers

Variance / Standard Deviation – Measures spread or dispersion

#### ii. Extraction

In [6]:
AGG_FUNCS = {
    "min":      np.min,
    "max":      np.max,
    "mean":     np.mean,
    "median":   np.median,
    "std":      np.std,
    "q1":       lambda x: np.quantile(x, 0.25),
    "q3":       lambda x: np.quantile(x, 0.75),
}

SENSOR_COLS = [
    "avg_rss12", "var_rss12",
    "avg_rss13", "var_rss13",
    "avg_rss23", "var_rss23",
]

def summarise_instance(df: pd.DataFrame) -> dict:
    feats = {}
    for col in SENSOR_COLS:
        for stat, fn in AGG_FUNCS.items():
            feats[f"{stat}-{col}"] = fn(df[col].values)  
    return feats

def make_feature_df(split_root: Path) -> pd.DataFrame:
    rows = []
    for csv in split_root.rglob("dataset*.csv"):
        raw     = load_csv(csv)
        feats   = summarise_instance(raw)
        feats["activity"] = csv.parent.name        
        feats["file"]     = csv.name               
        rows.append(feats)
    df = pd.DataFrame(rows)
    return df.sort_values(by=["activity", "file"]).reset_index(drop=True)

train_df = make_feature_df(TRAIN_ROOT)
test_df  = make_feature_df(TEST_ROOT)

print(f"Train: {train_df.shape}    Test: {test_df.shape}")
full_df   = pd.concat([train_df, test_df], ignore_index=True)
display(full_df)

Train: (69, 44)    Test: (19, 44)


,min-avg_rss12,max-avg_rss12,mean-avg_rss12,median-avg_rss12,std-avg_rss12,q1-avg_rss12,q3-avg_rss12,min-var_rss12,max-var_rss12,mean-var_rss12,...,q3-avg_rss23,min-var_rss23,max-var_rss23,mean-var_rss23,median-var_rss23,std-var_rss23,q1-var_rss23,q3-var_rss23,activity,file
0,35.00,47.40,43.954500,44.330,1.557210,43.00,45.0000,0.0,1.70,0.426250,...,36.500,0.0,1.79,0.493292,0.430,0.512971,0.000,0.940,bending1,dataset3.csv
1,33.00,47.75,42.179812,43.500,3.666840,39.15,45.0000,0.0,3.00,0.696042,...,36.330,0.0,2.18,0.613521,0.500,0.523771,0.000,1.000,bending1,dataset4.csv
2,33.00,45.75,41.678063,41.750,2.241152,41.33,42.7500,0.0,2.83,0.535979,...,31.250,0.0,1.79,0.383292,0.430,0.388759,0.000,0.500,bending1,dataset5.csv
3,37.00,48.00,43.454958,43.250,1.384653,42.50,45.0000,0.0,1.58,0.378083,...,24.000,0.0,5.26,0.679646,0.500,0.621885,0.430,0.870,bending1,dataset6.csv
4,36.25,48.00,43.969125,44.500,1.616677,43.31,44.6700,0.0,1.50,0.413125,...,23.750,0.0,2.96,0.555312,0.490,0.487318,0.000,0.830,bending1,dataset7.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,35.50,46.25,43.174938,43.670,1.986979,42.50,44.5000,0.0,2.12,0.506583,...,16.500,0.0,5.72,0.911979,0.830,0.665467,0.470,1.220,standing,dataset2.csv
84,32.75,47.00,42.760562,44.500,3.395376,41.33,45.3725,0.0,3.34,0.486167,...,18.565,0.0,5.73,0.842271,0.710,0.721413,0.430,1.090,standing,dataset3.csv
85,19.33,43.50,34.227771,35.500,4.884480,30.50,37.7500,0.0,14.50,3.995729,...,18.670,0.0,9.74,3.394125,3.100,1.790222,2.105,4.425,walking,dataset1.csv
86,12.50,45.00,33.509729,34.125,4.845868,30.50,36.7500,0.0,13.05,4.450771,...,18.750,0.0,8.96,3.378479,3.085,1.785497,2.060,4.440,walking,dataset2.csv


#### iii. Standard Deviation

In [7]:
feat_cols = [c for c in full_df.columns if "_" in c]

def bootstrap_ci(x, n_resamples=10_000, alpha=0.10, rng=None):
    rng = rng or np.random.default_rng()
    n   = len(x)
    resampled = rng.choice(x, size=(n_resamples, n), replace=True)
    stds = resampled.std(axis=1, ddof=1)               
    lower = np.percentile(stds, 100 * alpha / 2)
    upper = np.percentile(stds, 100 * (1 - alpha / 2))
    return x.std(ddof=1), lower, upper

ci_rows = []
for col in feat_cols:
    s, lo, hi = bootstrap_ci(full_df[col].values)
    ci_rows.append({"feature": col, "Standard Deviation": s, "Lower Bound of Confidence Interval": lo, "Upper Bound of Confidence Interval": hi})

ci_df = (pd.DataFrame(ci_rows).set_index("feature").round(4).sort_index())

display(ci_df)

,Standard Deviation,Lower Bound of Confidence Interval,Upper Bound of Confidence Interval
feature,,,
max-avg_rss12,4.3944,3.3203,5.2988
max-avg_rss13,4.8751,4.1523,5.4581
max-avg_rss23,5.7412,4.7329,6.5649
max-var_rss12,5.0627,4.6347,5.3992
max-var_rss13,2.1836,1.9709,2.3597
max-var_rss23,2.5189,2.2476,2.7635
mean-avg_rss12,5.3357,4.6957,5.8875
mean-avg_rss13,4.0084,3.4332,4.4993
mean-avg_rss23,5.6756,4.4085,6.7343


#### iv. Select Features

Min & Max gives the full amplitude (range) of the signal strength window and q3 will capture the upper-tail behaviour and will be less sensitive to outliers than max, giving complementary information.

## 2. ISLR 3.7.4

### (a) Linear Train

On the training set, the cubic model will almost always clock a smaller RSS. It’s packing in some extra polynomial terms, so it bends the fit more and naturally hoovers up more of the in-sample variance.

### (b) Linear Test

RSS will be more for cubic regression model when it comes to test data, because regression curve would overfit more in case of cubic model when compared to linear model.

### (c) Not Linear Train

For cubic regression RSS will be lower, because increasing the complexity reduces RSS in general.

### (d) Not Linear Testing

We cannot say for sure when it comes to test data, Because RSS might decrease or it might increase after increasing the complexity of the model... Knowing the true relationship is important in this case.

## 3. ISLR 3.7.3 - Extra Practice 

## 3. ISLR 3.7.5 - Extra Practice 